In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/Cranfield.zip -d /content/Cranfield

Archive:  /content/drive/MyDrive/Cranfield.zip
   creating: /content/Cranfield/Cranfield/
  inflating: /content/Cranfield/Cranfield/1.txt  
  inflating: /content/Cranfield/Cranfield/10.txt  
  inflating: /content/Cranfield/Cranfield/100.txt  
  inflating: /content/Cranfield/Cranfield/1000.txt  
  inflating: /content/Cranfield/Cranfield/1001.txt  
  inflating: /content/Cranfield/Cranfield/1002.txt  
  inflating: /content/Cranfield/Cranfield/1003.txt  
  inflating: /content/Cranfield/Cranfield/1004.txt  
  inflating: /content/Cranfield/Cranfield/1005.txt  
  inflating: /content/Cranfield/Cranfield/1006.txt  
  inflating: /content/Cranfield/Cranfield/1007.txt  
  inflating: /content/Cranfield/Cranfield/1008.txt  
  inflating: /content/Cranfield/Cranfield/1009.txt  
  inflating: /content/Cranfield/Cranfield/101.txt  
  inflating: /content/Cranfield/Cranfield/1010.txt  
  inflating: /content/Cranfield/Cranfield/1011.txt  
  inflating: /content/Cranfield/Cranfield/1012.txt  
  inflating: /co

In [4]:
!ls /content/Cranfield
!ls /content/Cranfield/Cranfield

Cranfield  TEST
1000.txt  1159.txt  1316.txt  212.txt  370.txt	528.txt  686.txt  843.txt
1001.txt  115.txt   1317.txt  213.txt  371.txt	529.txt  687.txt  844.txt
1002.txt  1160.txt  1318.txt  214.txt  372.txt	52.txt	 688.txt  845.txt
1003.txt  1161.txt  1319.txt  215.txt  373.txt	530.txt  689.txt  846.txt
1004.txt  1162.txt  131.txt   216.txt  374.txt	531.txt  68.txt   847.txt
1005.txt  1163.txt  1320.txt  217.txt  375.txt	532.txt  690.txt  848.txt
1006.txt  1164.txt  1321.txt  218.txt  376.txt	533.txt  691.txt  849.txt
1007.txt  1165.txt  1322.txt  219.txt  377.txt	534.txt  692.txt  84.txt
1008.txt  1166.txt  1323.txt  21.txt   378.txt	535.txt  693.txt  850.txt
1009.txt  1167.txt  1324.txt  220.txt  379.txt	536.txt  694.txt  851.txt
100.txt   1168.txt  1325.txt  221.txt  37.txt	537.txt  695.txt  852.txt
1010.txt  1169.txt  1326.txt  222.txt  380.txt	538.txt  696.txt  853.txt
1011.txt  116.txt   1327.txt  223.txt  381.txt	539.txt  697.txt  854.txt
1012.txt  1170.txt  1328.txt  224.txt 

In [4]:
!ls /content/Cranfield/TEST

query.txt  RES


In [5]:
!ls /content/Cranfield/Cranfield

1000.txt  1159.txt  1316.txt  212.txt  370.txt	528.txt  686.txt  843.txt
1001.txt  115.txt   1317.txt  213.txt  371.txt	529.txt  687.txt  844.txt
1002.txt  1160.txt  1318.txt  214.txt  372.txt	52.txt	 688.txt  845.txt
1003.txt  1161.txt  1319.txt  215.txt  373.txt	530.txt  689.txt  846.txt
1004.txt  1162.txt  131.txt   216.txt  374.txt	531.txt  68.txt   847.txt
1005.txt  1163.txt  1320.txt  217.txt  375.txt	532.txt  690.txt  848.txt
1006.txt  1164.txt  1321.txt  218.txt  376.txt	533.txt  691.txt  849.txt
1007.txt  1165.txt  1322.txt  219.txt  377.txt	534.txt  692.txt  84.txt
1008.txt  1166.txt  1323.txt  21.txt   378.txt	535.txt  693.txt  850.txt
1009.txt  1167.txt  1324.txt  220.txt  379.txt	536.txt  694.txt  851.txt
100.txt   1168.txt  1325.txt  221.txt  37.txt	537.txt  695.txt  852.txt
1010.txt  1169.txt  1326.txt  222.txt  380.txt	538.txt  696.txt  853.txt
1011.txt  116.txt   1327.txt  223.txt  381.txt	539.txt  697.txt  854.txt
1012.txt  1170.txt  1328.txt  224.txt  382.txt	53.txt	

In [6]:
!ls /content/Cranfield/TEST


query.txt  RES


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab') # Thêm dòng này vào
print("Setup complete.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Setup complete.


In [6]:
import os
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re
import operator
import sklearn.metrics
from statistics import mean

In [8]:
# Updated Paths to Cranfield files based on your structure
# Base directory for the unzipped content
BASE_CRANFIELD_DIR = '/content/Cranfield/Cranfield' # This is where your 000.txt, etc. are
QUERIES_FILE_PATH = '/content/Cranfield/TEST/query.txt'
RELEVANCE_DIR_PATH = '/content/Cranfield/TEST/RES' # This is a directory containing individual relevance files

In [12]:
def parse_cranfield_documents_custom(base_dir):
    documents_content_map = {} # Map Cranfield 1-based ID to document content

    # List all .txt files in the directory
    doc_filenames = [f for f in os.listdir(base_dir) if f.endswith('.txt')]

    print(f"Found {len(doc_filenames)} document files.")

    # Sort filenames numerically to ensure correct order
    doc_filenames.sort(key=lambda x: int(x.split('.')[0]))

    for filename in doc_filenames:
        file_path = os.path.join(base_dir, filename)
        try:
            # Assuming filename 'N.txt' means original Cranfield Doc ID N+1 (if starting from 0)
            # e.g., 000.txt -> Cranfield Doc ID 1, 001.txt -> Cranfield Doc ID 2
            doc_idx_from_filename = int(filename.split('.')[0])
            cranfield_doc_id = doc_idx_from_filename + 1

            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().strip()

            # Chỉ thêm vào map nếu nội dung không rỗng.
            # Đôi khi file có thể tồn tại nhưng trống.
            if content:
                documents_content_map[cranfield_doc_id] = content
            else:
                print(f"Warning: File {filename} (Cranfield ID {cranfield_doc_id}) is empty. Skipping.")

        except Exception as e:
            print(f"Error reading file {filename} (ID derived as {int(filename.split('.')[0]) + 1}): {e}")
            continue

    documents_list_for_vectorizer = [] # This will be the 'documents' variable for TF-IDF
    for i in range(1, 1401): # Cranfield has 1400 docs, IDs 1-1400
        if i in documents_content_map:
            documents_list_for_vectorizer.append(documents_content_map[i])
        else:
            print(f"Warning: Cranfield Document ID {i} is missing from loaded files or was empty. Filling with empty string.")
            documents_list_for_vectorizer.append("")

    return documents_list_for_vectorizer, documents_content_map # Return both for flexibility

print("Parsing documents...")
# 'documents' sẽ là list các chuỗi tài liệu, đây chính là documents_raw mà BM25 cần
documents_raw, docs_by_id = parse_cranfield_documents_custom(BASE_CRANFIELD_DIR)
documents = documents_raw # Đặt tên biến 'documents' để tương thích với TF-IDF của bạn
N_DOCS = len(documents)
print(f"Loaded {N_DOCS} documents.")

Parsing documents...
Found 1400 document files.
Loaded 1400 documents.


In [13]:
# --- Process Queries (query.txt) ---
def parse_cranfield_queries_custom(file_path):
    parsed_queries = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t', 1)
            if len(parts) == 2:
                query_id = int(parts[0])
                query_text = parts[1].strip()
                parsed_queries[query_id] = query_text
    return parsed_queries

print("Parsing queries...")
cranfield_queries_raw = parse_cranfield_queries_custom(QUERIES_FILE_PATH)
# Định nghĩa queries_list_sorted_ids ở đây để nó có thể truy cập được
queries_list_sorted_ids = sorted(cranfield_queries_raw.keys())
queries_text_list = [cranfield_queries_raw[q_id] for q_id in queries_list_sorted_ids]
N_QUERIES = len(queries_text_list)
print(f"Loaded {N_QUERIES} queries.")

Parsing queries...
Loaded 225 queries.


In [14]:
# --- Process Relevance Judgments (individual files in RES directory) ---
def parse_cranfield_relevance_custom(dir_path):
    relevance_judgments = {} # Dictionary: {query_id: {doc_id: relevance_score}}

    rel_filenames = [f for f in os.listdir(dir_path) if f.endswith('.txt')]

    for filename in rel_filenames:
        file_path = os.path.join(dir_path, filename)
        try:
            query_id = int(filename.split('.')[0])

            relevance_for_this_query = {}
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 3:
                        doc_id = int(parts[1])
                        relevance_score = int(parts[2])
                        relevance_for_this_query[doc_id] = relevance_score
            relevance_judgments[query_id] = relevance_for_this_query
        except Exception as e:
            print(f"Error reading relevance file {filename}: {e}")
            continue
    return relevance_judgments

print("Parsing relevance judgments...")
cranfield_relevance = parse_cranfield_relevance_custom(RELEVANCE_DIR_PATH)
print(f"Loaded relevance judgments for {len(cranfield_relevance)} queries.")

Parsing relevance judgments...
Loaded relevance judgments for 225 queries.


In [22]:
# --- Text Preprocessing Function ---
Stop_Words = stopwords.words("english")

def tokenize(text):
    clean_txt = re.sub('[^a-z\s]+',' ',text)
    clean_txt = re.sub('(\s+)',' ',clean_txt)
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(clean_txt))
    words = [word for word in words if word not in Stop_Words]
    words = filter(lambda t: len(t)>=min_length, words)
    tokens =(list(map(lambda token: PorterStemmer().stem(token),words)))
    return tokens

print("Tokenizer function defined.")

Tokenizer function defined.


In [25]:
VSM (TF-IDF) Vectorization - WITH HYPERPARAMETER TUNING

print("\n--- Starting TF-IDF Vectorizer Hyperparameter Tuning ---")

best_vsm_map = -1.0
best_vsm_params = {}
best_vsm_vectorizer = None
best_vsm_termdocmtx = None
best_vsm_features = None

max_dfs = [1.0, 0.95, 0.90, 0.85, 0.80] # Try different maximum document frequencies (1.0 means no max_df filter)

vsm_tuning_results = []

for current_min_df in min_dfs:
    for current_max_df in max_dfs:
        if current_max_df < 1.0 and current_min_df > int(current_max_df * N_DOCS):
             continue
        if current_min_df > N_DOCS:
            continue

        print(f"\nTesting VSM with min_df={current_min_df}, max_df={current_max_df}...")

        current_vectorizer = TfidfVectorizer(tokenizer=tokenize,
                                             min_df=current_min_df,
                                             max_df=current_max_df,
                                             use_idf=True, sublinear_tf=True, norm='l2')

        try:
            # Fit and transform
            current_termdocmtx = current_vectorizer.fit_transform(documents).toarray()
            current_features_ = current_vectorizer.get_feature_names_out()

            temp_vsm_avg_precisions = []

            for q_id in queries_list_sorted_ids:
                query_text = cranfield_queries_raw[q_id]
                relevant_doc_ids = {doc_id for doc_id, score in cranfield_relevance.get(q_id, {}).items() if score > 0}
                if not relevant_doc_ids:
                    continue

                ranked_docs_vsm = []
                try: # Ensure query can be transformed
                    qvec = current_vectorizer.transform([query_text]).toarray()
                    sim_scores = sklearn.metrics.pairwise.cosine_similarity(current_termdocmtx, qvec)

                    doc_sim = {}
                    for item in range(N_DOCS):
                        doc_sim[item + 1] = sim_scores[item][0]

                    ranked_docs_vsm = sorted([(doc_id, score) for doc_id, score in doc_sim.items() if score > 0],
                                             key=operator.itemgetter(1), reverse=True)
                except ValueError as e:
                    pass

                ap_vsm = calculate_average_precision(ranked_docs_vsm, relevant_doc_ids)
                temp_vsm_avg_precisions.append(ap_vsm)

            current_vsm_map = mean(temp_vsm_avg_precisions) if temp_vsm_avg_precisions else 0.0
            print(f"  VSM MAP with min_df={current_min_df}, max_df={current_max_df}: {current_vsm_map:.4f}")

            vsm_tuning_results.append({
                'min_df': current_min_df,
                'max_df': current_max_df,
                'map': current_vsm_map
            })

            if current_vsm_map > best_vsm_map:
                best_vsm_map = current_vsm_map
                best_vsm_params = {'min_df': current_min_df, 'max_df': current_max_df}
                best_vsm_vectorizer = current_vectorizer # Store the best vectorizer
                best_vsm_termdocmtx = current_termdocmtx # Store the best termdocmtx
                best_vsm_features = current_features_

        except Exception as e:
            print(f"  Error fitting VSM with min_df={current_min_df}, max_df={current_max_df}: {e}")
            continue

print("\n--- TF-IDF Vectorizer Hyperparameter Tuning Complete ---")
print(f"Best VSM MAP: {best_vsm_map:.4f} with parameters: {best_vsm_params}")

vectorizer = best_vsm_vectorizer
termdocmtx = best_vsm_termdocmtx
features_ = best_vsm_features # Update features_ as well if you use it later

if vectorizer is None:
    print("Warning: No suitable VSM vectorizer found. Falling back to default parameters.")
    # Fallback to a default if tuning failed completely
    vectorizer = TfidfVectorizer(tokenizer=tokenize, use_idf=True, sublinear_tf=True, norm='l2')
    termdocmtx = vectorizer.fit_transform(documents).toarray()
    features_ = vectorizer.get_feature_names_out()
    print(f"Default VSM Vectorizer parameters set to: min_df={vectorizer.min_df}, max_df={vectorizer.max_df}")

print(f"Final VSM Vectorizer parameters set to: min_df={vectorizer.min_df}, max_df={vectorizer.max_df}")
print(f"TF-IDF matrix created with shape: {termdocmtx.shape}")


--- Starting TF-IDF Vectorizer Hyperparameter Tuning ---

Testing VSM with min_df=1, max_df=1.0...
  VSM MAP with min_df=1, max_df=1.0: 0.1290

Testing VSM with min_df=1, max_df=0.95...


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  VSM MAP with min_df=1, max_df=0.95: 0.1290

Testing VSM with min_df=1, max_df=0.9...


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  VSM MAP with min_df=1, max_df=0.9: 0.1290

Testing VSM with min_df=1, max_df=0.85...


...

  VSM MAP with min_df=5, max_df=0.9: 0.1265

Testing VSM with min_df=5, max_df=0.85...


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  VSM MAP with min_df=5, max_df=0.85: 0.1265

Testing VSM with min_df=5, max_df=0.8...


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  VSM MAP with min_df=5, max_df=0.8: 0.1265

--- TF-IDF Vectorizer Hyperparameter Tuning Complete ---
Best VSM MAP: 0.1290 with parameters: {'min_df': 1, 'max_df': 1.0}
Final VSM Vectorizer parameters set to: min_df=1, max_df=1.0
TF-IDF matrix created with shape: (1400, 4211)


In [26]:
# --- KHỞI TẠO GLOBAL PORTER STEMMER
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()

print("\nTokenizing all documents and queries for consistent processing...")
documents_tokenized = [tokenize(doc) for doc in documents_raw]
# queries_list_sorted_ids được dùng để đảm bảo thứ tự khi truy vấn
queries_tokenized_dict = {q_id: tokenize(q_text) for q_id, q_text in cranfield_queries_raw.items()}
print("Document and Query tokenization complete for BM25.")


# Cell 7: BM25 Class Implementation and Model Fitting - WITH HYPERPARAMETER TUNING
print("\nDefining BM25 class and starting hyperparameter tuning...")

class BM25:
    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus_tokenized): # Takes corpus already tokenized
        tf = []
        df = {}
        doc_len = []
        corpus_size = 0
        for document_tokens in corpus_tokenized:
            corpus_size += 1
            doc_len.append(len(document_tokens))

            frequencies = {}
            for term in document_tokens:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count
            tf.append(frequencies)

            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        self.idf_ = {}
        for term, freq in df.items():
            self.idf_[term] = math.log((corpus_size - freq + 0.5) / (freq + 0.5))
            self.idf_[term] = max(0, self.idf_[term])

        self.tf_ = tf
        self.df_ = df
        self.doc_len_ = doc_len
        self.corpus_tokenized_ = corpus_tokenized
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query_tokens): # Takes query already tokenized
        scores = [self._score(query_tokens, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query_tokens, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query_tokens:
            if term not in frequencies or term not in self.idf_:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

# --- BM25 Hyperparameter Tuning ---
best_bm25_map = -1.0
best_bm25_params = {}

k1_values = [0.8, 1.0, 1.2, 1.5, 1.8, 2.0] # Common range for k1
b_values = [0.6, 0.7, 0.75, 0.8, 0.9] # Common range for b

bm25_tuning_results = []

for current_k1 in k1_values:
    for current_b in b_values:
        print(f"\nTesting BM25 with k1={current_k1}, b={current_b}...")

        current_bm25_model = BM25(k1=current_k1, b=current_b)
        current_bm25_model.fit(documents_tokenized)

        # --- Evaluate BM25 with current parameters (mini-evaluation) ---
        temp_bm25_avg_precisions = []

        for q_id in queries_list_sorted_ids:
            query_tokens = queries_tokenized_dict[q_id]
            relevant_doc_ids = {doc_id for doc_id, score in cranfield_relevance.get(q_id, {}).items() if score > 0}
            if not relevant_doc_ids:
                continue

            ranked_docs_bm25 = current_bm25_model.search(query_tokens)
            # BM25 search returns scores directly, need to convert to (doc_id, score) pairs
            bm25_ranked_list = sorted([(idx + 1, score) for idx, score in enumerate(ranked_docs_bm25) if score > 0],
                                      key=operator.itemgetter(1), reverse=True)

            ap_bm25 = calculate_average_precision(bm25_ranked_list, relevant_doc_ids)
            temp_bm25_avg_precisions.append(ap_bm25)

        current_bm25_map = mean(temp_bm25_avg_precisions) if temp_bm25_avg_precisions else 0.0
        print(f"  BM25 MAP with k1={current_k1}, b={current_b}: {current_bm25_map:.4f}")

        bm25_tuning_results.append({
            'k1': current_k1,
            'b': current_b,
            'map': current_bm25_map
        })

        if current_bm25_map > best_bm25_map:
            best_bm25_map = current_bm25_map
            best_bm25_params = {'k1': current_k1, 'b': current_b}
            bm25_model = current_bm25_model # Store the best BM25 model

print("\n--- BM25 Hyperparameter Tuning Complete ---")
print(f"Best BM25 MAP: {best_bm25_map:.4f} with parameters: {best_bm25_params}")

if bm25_model is None:
    print("Warning: No suitable BM25 model found. Falling back to default parameters (k1=1.5, b=0.75).")
    bm25_model = BM25()
    bm25_model.fit(documents_tokenized)
print(f"Final BM25 Model parameters set to: k1={bm25_model.k1}, b={bm25_model.b}")


Tokenizing all documents and queries for consistent processing...
Document and Query tokenization complete for BM25.

Defining BM25 class and starting hyperparameter tuning...

Testing BM25 with k1=0.8, b=0.6...
  BM25 MAP with k1=0.8, b=0.6: 0.1273

Testing BM25 with k1=0.8, b=0.7...
  BM25 MAP with k1=0.8, b=0.7: 0.1304

Testing BM25 with k1=0.8, b=0.75...
  BM25 MAP with k1=0.8, b=0.75: 0.1304

Testing BM25 with k1=0.8, b=0.8...
  BM25 MAP with k1=0.8, b=0.8: 0.1304

Testing BM25 with k1=0.8, b=0.9...
  BM25 MAP with k1=0.8, b=0.9: 0.1323

Testing BM25 with k1=1.0, b=0.6...
  BM25 MAP with k1=1.0, b=0.6: 0.1310

Testing BM25 with k1=1.0, b=0.7...
  BM25 MAP with k1=1.0, b=0.7: 0.1323

Testing BM25 with k1=1.0, b=0.75...
  BM25 MAP with k1=1.0, b=0.75: 0.1316

Testing BM25 with k1=1.0, b=0.8...
  BM25 MAP with k1=1.0, b=0.8: 0.1321

Testing BM25 with k1=1.0, b=0.9...
  BM25 MAP with k1=1.0, b=0.9: 0.1330

Testing BM25 with k1=1.2, b=0.6...
  BM25 MAP with k1=1.2, b=0.6: 0.1318

Test

In [32]:
# --- Interactive Query ---
# This part allows you to input a query and get results.
query_input = input("Enter your query: \n")
qvec_input = vectorizer.transform([query_input]).toarray()

# Calculate cosine similarity between query vector and all document vectors
sim_scores_input = sklearn.metrics.pairwise.cosine_similarity(termdocmtx, qvec_input)

# Store similarity scores in a dictionary: {document_index: score}
doc_sim_input = {}
for item in range(N_DOCS): # N_DOCS is 1400
    doc_sim_input[item] = sim_scores_input[item][0] # Access the scalar score

# Sort documents by similarity score in descending order
sort_dict_input = sorted(doc_sim_input.items(), key=operator.itemgetter(1), reverse=True)

# Ask for the number of results
try:
    k_input = int(input("Enter the number of results expected: \n"))
except ValueError:
    k_input = 10 # Default to 10 if input is invalid
    print(f"Invalid input, defaulting to {k_input} results.")

# Get top-k document indices (0-based)
results_indices_input = [sort_dict_input[i][0] for i in range(min(k_input, N_DOCS))]

# Retrieve the actual document content for the top-k results
# Remember 'documents' list is 0-indexed, corresponding to Cranfield IDs 1-1400
results_topk_content = [documents[u] for u in results_indices_input]

print("\n--- Top Results for your query ---")
for i, doc_idx in enumerate(results_indices_input):
    cranfield_doc_id = doc_idx + 1 # Convert 0-based list index back to 1-based Cranfield ID
    print(f"Rank {i+1}: Document ID {cranfield_doc_id} (Score: {doc_sim_input[doc_idx]:.4f})")
    print(f"Content (first 200 chars): {documents[doc_idx][:200]}...\n")

Enter your query: 
how to code
Enter the number of results expected: 
3

--- Top Results for your query ---
Rank 1: Document ID 1135 (Score: 0.3512)
Content (first 200 chars): torispherical shells   a caution to designers . it has recently become apparent, through a rigorous stress analysis of a specific case that designing torispherical shells by the current edition of the...

Rank 2: Document ID 238 (Score: 0.3264)
Content (first 200 chars): a compressor routine test code . the routine testing of aircraft type compressors.dash in the main, axial flow, multi stage compressors.dash requires a compromise between research accuracy and the pra...

Rank 3: Document ID 1 (Score: 0.0000)
Content (first 200 chars): ...



In [28]:
# Cell 9: Evaluation Metric Functions - UPDATED & CORRECTED

print("Defining evaluation metric functions...")

def calculate_precision_recall(retrieved_doc_ids, relevant_doc_ids):
    """
    Calculates Precision and Recall for a single query.
    Args:
        retrieved_doc_ids (set): Set of IDs of retrieved documents.
        relevant_doc_ids (set): Set of IDs of truly relevant documents.
    Returns:
        tuple: (precision, recall)
    """
    num_relevant_retrieved = len(retrieved_doc_ids.intersection(relevant_doc_ids))

    precision = num_relevant_retrieved / len(retrieved_doc_ids) if len(retrieved_doc_ids) > 0 else 0
    recall = num_relevant_retrieved / len(relevant_doc_ids) if len(relevant_doc_ids) > 0 else 0

    return precision, recall

def calculate_average_precision(retrieved_rankings, relevant_doc_ids):
    """
    Calculates Average Precision (AP) for a single query.
    Args:
        retrieved_rankings (list): List of (doc_id, score) tuples, sorted by score.
                                   doc_id should be 1-based.
        relevant_doc_ids (set): Set of IDs of truly relevant documents (1-based).
    Returns:
        float: Average Precision for the query.
    """
    if not relevant_doc_ids:
        return 0.0

    sum_precisions = 0.0
    num_relevant_found = 0

    for i, (doc_id, _) in enumerate(retrieved_rankings):
        if doc_id in relevant_doc_ids: # ĐÃ SỬA LỖI: relevant_doc_found -> relevant_doc_ids
            num_relevant_found += 1
            # Precision at k: (number of relevant documents found up to rank k) / (k)
            precision_at_k = num_relevant_found / (i + 1)
            sum_precisions += precision_at_k

    return sum_precisions / len(relevant_doc_ids)


def calculate_interpolated_precision(retrieved_rankings, relevant_doc_ids, num_recall_points=11):
    """
    Calculates 11-point interpolated precision for a single query.
    Args:
        retrieved_rankings (list): List of (doc_id, score) tuples, sorted by score.
                                   doc_id should be 1-based.
        relevant_doc_ids (set): Set of IDs of truly relevant documents (1-based).
        num_recall_points (int): Number of recall points (e.g., 11 for 0.0, 0.1, ..., 1.0).
    Returns:
        list: Interpolated precision values at each recall point.
    """
    if not relevant_doc_ids:
        return [0.0] * num_recall_points

    actual_relevant_count = len(relevant_doc_ids)

    # Calculate precision and recall at each retrieved relevant document
    prec_rec_points = [] # List of (recall, precision)
    num_relevant_found = 0

    # Iterate through ranked documents to find precision/recall points
    # Only consider points where a relevant document is found
    for i, (doc_id, _) in enumerate(retrieved_rankings):
        if doc_id in relevant_doc_ids:
            num_relevant_found += 1
            recall = num_relevant_found / actual_relevant_count
            precision = num_relevant_found / (i + 1)
            prec_rec_points.append((recall, precision))

    # Initialize interpolated precision values
    interpolated_precisions = [0.0] * num_recall_points

    # Standard recall levels (0.0, 0.1, ..., 1.0)
    recall_levels = [j / (num_recall_points - 1) for j in range(num_recall_points)]

    # Calculate interpolated precision for each standard recall level
    for j, r_level in enumerate(recall_levels):
        max_prec_for_r = 0.0
        # For each r_level, find the maximum precision at any recall r' >= r_level
        for rec_val, prec_val in prec_rec_points:
            if rec_val >= r_level:
                max_prec_for_r = max(max_prec_for_r, prec_val)
        interpolated_precisions[j] = max_prec_for_r

    return interpolated_precisions

print("Evaluation metric functions defined.")

Defining evaluation metric functions...
Evaluation metric functions defined.


In [29]:
# Cell 10: Comprehensive Evaluation Loop and Results Display

print("\n--- Starting Comprehensive Evaluation for VSM and BM25 ---")

# Dictionaries to store results for each model
results_vsm = {
    'precision': [],
    'recall': [],
    'avg_precision': [],
    'interpolated_precision_points': [0.0] * 11 # Initialize 11 points for summation
}

results_bm25 = {
    'precision': [],
    'recall': [],
    'avg_precision': [],
    'interpolated_precision_points': [0.0] * 11
}

evaluated_query_count = 0

# Loop through each query using its sorted ID (from Cell 4)
for q_id in queries_list_sorted_ids:
    query_text = cranfield_queries_raw[q_id] # Raw query string for VSM
    query_tokens = queries_tokenized_dict[q_id] # Tokenized query for BM25

    # Get relevant documents for the current query (1-based IDs)
    relevant_doc_ids = {doc_id for doc_id, score in cranfield_relevance.get(q_id, {}).items() if score > 0}

    if not relevant_doc_ids:
        # print(f"Warning: No relevant documents found for query {q_id}. Skipping evaluation for this query.")
        continue # Skip queries that have no relevant documents defined

    evaluated_query_count += 1

    # --- Evaluate VSM ---
    # get_ranked_documents returns a list of (doc_id, score), where doc_id is 1-based
    ranked_docs_vsm = get_ranked_documents('VSM', query_tokens=None, query_text=query_text)

    # Get the set of retrieved document IDs (1-based)
    retrieved_vsm_all_ids = {doc_id for doc_id, score in ranked_docs_vsm}

    # Calculate Precision and Recall
    p_vsm, r_vsm = calculate_precision_recall(retrieved_vsm_all_ids, relevant_doc_ids)
    results_vsm['precision'].append(p_vsm)
    results_vsm['recall'].append(r_vsm)

    # Calculate Average Precision (AP)
    ap_vsm = calculate_average_precision(ranked_docs_vsm, relevant_doc_ids)
    results_vsm['avg_precision'].append(ap_vsm)

    # Calculate 11-point Interpolated Precision and sum it up
    ip_vsm = calculate_interpolated_precision(ranked_docs_vsm, relevant_doc_ids)
    for i in range(11):
        results_vsm['interpolated_precision_points'][i] += ip_vsm[i]


    # --- Evaluate BM25 ---
    ranked_docs_bm25 = get_ranked_documents('BM25', query_tokens=query_tokens, query_text=None)
    retrieved_bm25_all_ids = {doc_id for doc_id, score in ranked_docs_bm25}

    p_bm25, r_bm25 = calculate_precision_recall(retrieved_bm25_all_ids, relevant_doc_ids)
    results_bm25['precision'].append(p_bm25)
    results_bm25['recall'].append(r_bm25)

    ap_bm25 = calculate_average_precision(ranked_docs_bm25, relevant_doc_ids)
    results_bm25['avg_precision'].append(ap_bm25)

    ip_bm25 = calculate_interpolated_precision(ranked_docs_bm25, relevant_doc_ids)
    for i in range(11):
        results_bm25['interpolated_precision_points'][i] += ip_bm25[i]


# --- Aggregate and Display Results ---
print("\n--- Summary Evaluation Results Across All Queries ---")

# VSM Summary
mean_p_vsm = mean(results_vsm['precision']) if results_vsm['precision'] else 0
mean_r_vsm = mean(results_vsm['recall']) if results_vsm['recall'] else 0
map_vsm = mean(results_vsm['avg_precision']) if results_vsm['avg_precision'] else 0
mean_11_point_vsm = [p / evaluated_query_count for p in results_vsm['interpolated_precision_points']] if evaluated_query_count > 0 else [0.0] * 11

print("\n--- Vector Space Model (VSM) ---")
print(f"Mean Precision: {mean_p_vsm:.4f}")
print(f"Mean Recall: {mean_r_vsm:.4f}")
print(f"Mean Average Precision (MAP): {map_vsm:.4f}")
print("11-Point Interpolated Precision (Recall Levels 0.0 to 1.0):")
for i, p_val in enumerate(mean_11_point_vsm):
    print(f"  Recall {i*0.1:.1f}: {p_val:.4f}")

# BM25 Summary
mean_p_bm25 = mean(results_bm25['precision']) if results_bm25['precision'] else 0
mean_r_bm25 = mean(results_bm25['recall']) if results_bm25['recall'] else 0
map_bm25 = mean(results_bm25['avg_precision']) if results_bm25['avg_precision'] else 0
mean_11_point_bm25 = [p / evaluated_query_count for p in results_bm25['interpolated_precision_points']] if evaluated_query_count > 0 else [0.0] * 11

print("\n--- BM25 Model ---")
print(f"Mean Precision: {mean_p_bm25:.4f}")
print(f"Mean Recall: {mean_r_bm25:.4f}")
print(f"Mean Average Precision (MAP): {map_bm25:.4f}")
print("11-Point Interpolated Precision (Recall Levels 0.0 to 1.0):")
for i, p_val in enumerate(mean_11_point_bm25):
    print(f"  Recall {i*0.1:.1f}: {p_val:.4f}")


print("\n--- MODEL COMPARISON (MAP) ---")
print(f"VSM Mean Average Precision (MAP): {map_vsm:.4f}")
print(f"BM25 Mean Average Precision (MAP): {map_bm25:.4f}")

# Create DataFrame for 11-point interpolated precision comparison
recall_levels = [f"{i*0.1:.1f}" for i in range(11)]
comparison_df = pd.DataFrame({
    'Recall Level': recall_levels,
    'VSM Precision': [f"{p:.4f}" for p in mean_11_point_vsm],
    'BM25 Precision': [f"{p:.4f}" for p in mean_11_point_bm25]
})
print("\n11-Point Interpolated Precision Comparison:")
print(comparison_df.to_string(index=False))

print("\n--- Comprehensive Evaluation Complete ---")


--- Starting Comprehensive Evaluation for VSM and BM25 ---

--- Summary Evaluation Results Across All Queries ---

--- Vector Space Model (VSM) ---
Mean Precision: 0.0072
Mean Recall: 0.8426
Mean Average Precision (MAP): 0.1290
11-Point Interpolated Precision (Recall Levels 0.0 to 1.0):
  Recall 0.0: 0.3460
  Recall 0.1: 0.3250
  Recall 0.2: 0.2623
  Recall 0.3: 0.1963
  Recall 0.4: 0.1577
  Recall 0.5: 0.1325
  Recall 0.6: 0.0654
  Recall 0.7: 0.0427
  Recall 0.8: 0.0298
  Recall 0.9: 0.0123
  Recall 1.0: 0.0102

--- BM25 Model ---
Mean Precision: 0.0074
Mean Recall: 0.8261
Mean Average Precision (MAP): 0.1358
11-Point Interpolated Precision (Recall Levels 0.0 to 1.0):
  Recall 0.0: 0.3627
  Recall 0.1: 0.3388
  Recall 0.2: 0.2814
  Recall 0.3: 0.2119
  Recall 0.4: 0.1679
  Recall 0.5: 0.1446
  Recall 0.6: 0.0688
  Recall 0.7: 0.0430
  Recall 0.8: 0.0301
  Recall 0.9: 0.0125
  Recall 1.0: 0.0099

--- MODEL COMPARISON (MAP) ---
VSM Mean Average Precision (MAP): 0.1290
BM25 Mean Averag

In [30]:
# Cell 11: Implementing a Semantic Search Model (e.g., using Sentence Embeddings) for Bonus Points

print("\n--- Implementing Semantic Search (Sentence Embeddings) for Bonus ---")

# First, install the library if you haven't
# Chạy dòng này nếu bạn chưa cài đặt thư viện 'sentence-transformers'
# !pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np

print("Loading SentenceTransformer model... This might take a few minutes for the first time.")
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
print("SentenceTransformer model loaded.")

# --- Indexing Documents (creating embeddings) ---
print("Creating document embeddings (This might take a while)...")
# documents_raw contains the raw text from Cell 4
document_embeddings = semantic_model.encode(documents_raw, show_progress_bar=True)
print(f"Document embeddings created. Shape: {document_embeddings.shape}")

# --- Evaluating Semantic Search ---
print("\n--- Starting Evaluation for Semantic Search Model ---")

semantic_results = {
    'precision': [],
    'recall': [],
    'avg_precision': [],
    'interpolated_precision_points':
}

for q_id in queries_list_sorted_ids:
    query_text = cranfield_queries_raw[q_id]
    relevant_doc_ids = {doc_id for doc_id, score in cranfield_relevance.get(q_id, {}).items() if score > 0}

    if not relevant_doc_ids:
        continue

    query_embedding = semantic_model.encode([query_text])[0] # Encode query as a single vector
    semantic_scores = sklearn.metrics.pairwise.cosine_similarity(query_embedding.reshape(1, -1), document_embeddings)[0]

    doc_sim_semantic = {}
    for item in range(N_DOCS):
        doc_sim_semantic[item + 1] = semantic_scores[item] # Use 1-based Cranfield ID

    ranked_docs_semantic = sorted([(doc_id, score) for doc_id, score in doc_sim_semantic.items() if score > 0],
                                  key=operator.itemgetter(1), reverse=True)

    # Calculate metrics
    retrieved_semantic_all_ids = {doc_id for doc_id, score in ranked_docs_semantic}

    p_sem, r_sem = calculate_precision_recall(retrieved_semantic_all_ids, relevant_doc_ids)
    semantic_results['precision'].append(p_sem)
    semantic_results['recall'].append(r_sem)

    ap_sem = calculate_average_precision(ranked_docs_semantic, relevant_doc_ids)
    semantic_results['avg_precision'].append(ap_sem)

    ip_sem = calculate_interpolated_precision(ranked_docs_semantic, relevant_doc_ids)
    for i in range(11):
        semantic_results['interpolated_precision_points'][i] += ip_sem[i]

if 'evaluated_query_count' not in locals():
    # Fallback if evaluated_query_count is not globally defined from Cell 10
    evaluated_query_count = 0
    for q_id in queries_list_sorted_ids:
        if {doc_id for doc_id, score in cranfield_relevance.get(q_id, {}).items() if score > 0}:
            evaluated_query_count += 1
    if evaluated_query_count == 0:
        print("Warning: No queries with relevant documents found for evaluation.")

mean_p_sem = mean(semantic_results['precision']) if semantic_results['precision'] else 0
mean_r_sem = mean(semantic_results['recall']) if semantic_results['recall'] else 0
map_sem = mean(semantic_results['avg_precision']) if semantic_results['avg_precision'] else 0
mean_11_point_sem = [p / evaluated_query_count for p in semantic_results['interpolated_precision_points']] if evaluated_query_count > 0 else [0.0] * 11

print("\n--- Semantic Search Model (Sentence Embeddings) ---")
print(f"Mean Precision: {mean_p_sem:.4f}")
print(f"Mean Recall: {mean_r_sem:.4f}")
print(f"Mean Average Precision (MAP): {map_sem:.4f}")
print("11-Point Interpolated Precision (Recall Levels 0.0 to 1.0):")
for i, p_val in enumerate(mean_11_point_sem):
    print(f"  Recall {i*0.1:.1f}: {p_val:.4f}")

print("\n--- Semantic Search Evaluation Complete ---")



--- Implementing Semantic Search (Sentence Embeddings) for Bonus ---
Loading SentenceTransformer model... This might take a few minutes for the first time.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer model loaded.
Creating document embeddings (This might take a while)...


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Document embeddings created. Shape: (1400, 384)

--- Starting Evaluation for Semantic Search Model ---

--- Semantic Search Model (Sentence Embeddings) ---
Mean Precision: 0.0052
Mean Recall: 0.9840
Mean Average Precision (MAP): 0.1269
11-Point Interpolated Precision (Recall Levels 0.0 to 1.0):
  Recall 0.0: 0.3307
  Recall 0.1: 0.3168
  Recall 0.2: 0.2500
  Recall 0.3: 0.1930
  Recall 0.4: 0.1547
  Recall 0.5: 0.1280
  Recall 0.6: 0.0684
  Recall 0.7: 0.0468
  Recall 0.8: 0.0384
  Recall 0.9: 0.0188
  Recall 1.0: 0.0157

--- Semantic Search Evaluation Complete ---
